
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

### Install required libraries

In [1]:
%tensorflow_version 1.x
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 

!git clone https://github.com/tensorflow/models.git && (cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc)
!rsync -avq /content/models  {BASE_DIR}/ && rm -r /content/models
%env PYTHONPATH=:{BASE_DIR}/models/:{BASE_DIR}/models/research/slim'

TensorFlow 1.x selected.
--2020-06-11 07:48:13--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4958 (4.8K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   4.84K  --.-KB/s    in 0s      

2020-06-11 07:48:14 (60.0 MB/s) - ‘constants.py’ saved [4958/4958]

Cloning into 'models'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 36813 (delta 1), reused 6 (delta 0), pack-reused 36806
Receiving objects: 100% (36813/36813), 520.49 MiB | 32.95 MiB/s, done.
Resolving deltas: 100% (24588/24588), done.
Checking out files: 100% (2339/2339), d

In [2]:
#EDGE TPU REQUIREMENTS
#!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
#!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
#!sudo apt-get update
#!sudo apt-get install edgetpu-compiler

#!apt-get install protobuf-compiler python-tk 
!apt-get install python-pil python-lxml 
!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install matplotlib
!pip install tf_slim

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 32 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-pkg-resources all 39.0.1-2 [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-chardet all 3.0.

In [4]:
%cd {BASE_DIR}/models/research
!python setup.py build
!python setup.py install
!protoc object_detection/protos/*.proto --python_out=.

/content/tensorflow/models/research
running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detect

### Prepare dataset ###

#### Connect to Google Drive

In [5]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


TIMEOUT: ignored

In [0]:
#Download model config and save to to config directory
!wget {pretrained_model_key['config_url']} ‐‐directory-prefix={CONFIG_DIR} --{pretrained_model_key['name']}

#### Copy dataset from Google Drive and extract

In [9]:
!gsutil cp {DRIVE_DEV}model_graph.zip /content/
!unzip -qq model_graph.zip "trained_inference_graphs/*" "{INF_GRAPH}/"

!gsutil cp {DRIVE_DEV}model_checkpoints.zip /content/
!unzip -qq model_checkpoints.zip "model/*" "{CKPT_DIR}/"


CommandException: No URLs matched: /content/drive/My Drive/Machine Learning/Projects/Face-Mask-Detection/Dev/model_graph.zip
unzip:  cannot find or open model_graph.zip, model_graph.zip.zip or model_graph.zip.ZIP.


### Export inference graph

In [0]:
!python export_inference_graph.py \
  --pipeline_config_path="{CONFIG_DIR}/{pretrained_model_key['name']}" \
  --trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-22564" \
  --output_directory="{INF_GRAPH}" \
  --add_postprocessing_op=true \
  --input_type=image_tensor \
  

### Export TF Lite SSD inference graph

In [9]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="/content/training_demo/{pretrained_model_key['name']}" \
--trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-18560" \
--output_directory="{TF_LITE_GRAPH}" \
--add_postprocessing_op=true

Traceback (most recent call last):
  File "/content/models/research/object_detection/export_tflite_ssd_graph.py", line 96, in <module>
    from object_detection import export_tflite_ssd_graph_lib
  File "/content/models/research/object_detection/export_tflite_ssd_graph_lib.py", line 26, in <module>
    from tensorflow.tools.graph_transforms import TransformGraph
ModuleNotFoundError: No module named 'tensorflow.tools.graph_transforms'


### Convert graph to TF Lite

In [0]:
!tflite_convert \
--graph_def_file="/content/training_demo/tflite_model/tflite_graph.pb" \
--output_file="/content/training_demo/tflite_model/detect.tflite" \
--output_format=TFLITE \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=127 \
--change_concat_input_ranges=false \
--allow_custom_ops

2020-06-09 16:30:12.775293: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-09 16:30:12.777095: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-06-09 16:30:12.777140: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b2ec1e9bfca0): /proc/driver/nvidia/version does not exist
2020-06-09 16:30:12.777502: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-06-09 16:30:12.783186: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000165000 Hz
2020-06-09 16:30:12.783450: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1894a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-09 16:30:1

## Save 

### Archive the inference graph

In [0]:
!zip -r tflite-graphv1.zip {INF_GRAPH}
!gsutil cp tflite-graphv1.zip {DRIVE_DEV}

/content/training_demo
--2020-06-05 08:59:06--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4766 (4.7K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.65K  --.-KB/s    in 0s      

2020-06-05 08:59:06 (46.5 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4766/4766]



In [0]:
!zip -r inf-graphv1.zip {TF_LITE_GRAPH}
!gsutil cp inf-graphv1.zip {DRIVE_DEV}

### Save archive to Google Drive

/content/training_demo/images/test
185
